<a href="https://colab.research.google.com/github/SlLeonn/SenalesySistemas/blob/main/Ejercicios_SLIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***1_Convolucion***

### Evaluación de la convolución $ y(t) = x(t) * h(t) $

Dadas las señales:

- $ x(t) = \text{rect}\left(t - \frac{1}{2}\right) = u(t) - u(t - 1) $
- $ h(t) = e^{-t} u(t) $

La convolución se define como:

$$
y(t) = \int_{-\infty}^{\infty} x(\tau) h(t - \tau) \, d\tau
$$

---

#### **Caso 1: $ t < 0 $**

No hay traslape entre $ x(\tau) $ y $ h(t - \tau) $, entonces:

$$
y(t) = 0
$$

---

#### **Caso 2: $ 0 \leq t < 1 $**

El intervalo de integración es desde \( \tau = 0 \) hasta \( \tau = t \):

$$
y(t) = \int_0^t e^{-(t - \tau)} \, d\tau = e^{-t} \int_0^t e^{\tau} \, d\tau = e^{-t} (e^t - 1) = 1 - e^{-t}
$$

---

#### **Caso 3: $ t \geq 1 $**

Ahora el traslape ocurre completamente en \( \tau \in [0, 1] \):

$$
y(t) = \int_0^1 e^{-(t - \tau)} \, d\tau = e^{-t} \int_0^1 e^{\tau} \, d\tau = e^{-t}(e - 1)
$$

---

### Resultado final:

$$
y(t) =
\begin{cases}
0, & t < 0 \\\\
1 - e^{-t}, & 0 \leq t < 1 \\\\
(e - 1)e^{-t}, & t \geq 1
\end{cases}
$$


### ***2_Respuesta_Impulso***

---
- Compare la señal de salida obtenida al resolver la EDO con la obtenida mediante la convolución. Son iguales?
---

In [ ]:
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
from sympy import Heaviside, DiracDelta

# Configurar impresión bonita
sym.init_printing()

# Variables simbólicas
t, tau = sym.symbols('t tau', real=True)
x = sym.exp(-2*t) * sym.Heaviside(t)

# EDO: y' + y = x(t)
y = sym.Function('y')(t)
ode = sym.Eq(y.diff(t) + y, x)
sol_edo = sym.dsolve(ode)
C1 = sym.symbols('C1')
C1_val = sym.solve(sol_edo.rhs.subs(t, 0), C1)[0]
y_edo = sol_edo.rhs.subs(C1, C1_val)

# Mostrar solución simbólica de la EDO
display(sym.Eq(sym.Function('y')(t), y_edo.simplify()))

# Respuesta al impulso
h = sym.Function('h')(t)
impulse_ode = sym.Eq(h.diff(t) + h, DiracDelta(t))
sol_h = sym.dsolve(impulse_ode)
C2 = sym.symbols('C1')
C2_val = sym.solve(sol_h.rhs.limit(t, 0, '-'), C2)[0]
h_t = sol_h.rhs.subs(C2, C2_val)

# Mostrar respuesta al impulso h(t)
display(sym.Eq(sym.Function('h')(t), h_t.simplify()))

# Convolución: y(t) = ∫₀ᵗ x(τ) h(t - τ) dτ
x_tau = sym.exp(-2*tau)
h_shifted = h_t.subs(t, t - tau).subs(Heaviside(t), 1)
y_conv = sym.integrate(x_tau * h_shifted, (tau, 0, t))

# Mostrar resultado de la convolución simbólicamente
display(sym.Eq(sym.Function('y')(t), y_conv.simplify()))

# Funciones numéricas
y_edo_func = sym.lambdify(t, y_edo, 'numpy')
y_conv_func = sym.lambdify(t, y_conv, 'numpy')

# Tiempo y gráficos
t_vals = np.linspace(0, 10, 400)

plt.figure(figsize=(8, 4))
plt.plot(t_vals, y_edo_func(t_vals), label='y(t) por EDO', color='blue')
plt.plot(t_vals, y_conv_func(t_vals), label='y(t) por Convolución', color='red', linestyle='--')
plt.xlabel('t')
plt.ylabel('y(t)')
plt.title('Comparación de la salida: EDO vs Convolución')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

---
- Compruebe la solución $ h(t) $  de la EDO cuando $ x(t)=δ(t) $ de manera manual. Tener en cuenta que  $ (d/dt) $ $ ϵ(t)=δ(t) $.
---

### Verificación manual de la solución $ h(t) = e^{-t} u(t) $ para la EDO

Queremos comprobar que la función

$$
h(t) = e^{-t} u(t)
$$

satisface la ecuación diferencial:

$$
\frac{d}{dt} h(t) + h(t) = \delta(t)
$$

donde $ u(t) $ es la función escalón (Heaviside), y se cumple:

$$
\frac{d}{dt} u(t) = \delta(t)
$$

---

#### Paso 1: Derivada de $ h(t) $

Aplicando la regla del producto:

$$
\frac{d}{dt} h(t) = \frac{d}{dt} \left( e^{-t} u(t) \right)
= \left( \frac{d}{dt} e^{-t} \right) u(t) + e^{-t} \frac{d}{dt} u(t)
= -e^{-t} u(t) + e^{-t} \delta(t)
$$

---

#### Paso 2: Sustituimos en la EDO

Sumamos $ \frac{d}{dt} h(t) + h(t) $:

$$
\left( -e^{-t} u(t) + e^{-t} \delta(t) \right) + e^{-t} u(t)
= e^{-t} \delta(t)
$$

---

#### Paso 3: Evaluar en \( t = 0 \)

Como $ e^{-t} \delta(t) = \delta(t) $, porque la delta "extrae" el valor en $ t = 0 $:

$$
e^{-t} \delta(t) = e^0 \cdot \delta(t) = \delta(t)
$$

---

### ✅ Conclusión

Hemos comprobado que:

$$
\frac{d}{dt} h(t) + h(t) = \delta(t)
$$

por lo tanto, $ h(t) = e^{-t} u(t) $ **es solución** de la EDO con entrada impulso.


---
- Comprobar la solución de la integral de convolución de manera manual. Tener en cuenta las funciones Heaviside
---

### Verificación manual de la solución de la convolución

Queremos comprobar que la convolución:

$$
y(t) = x(t) * h(t) = \int_0^t x(\tau) \cdot h(t - \tau) \, d\tau
$$

con las señales:

- Entrada:
  $$
  x(t) = e^{-2t} u(t)
  $$

- Respuesta al impulso:
  $$
  h(t) = e^{-t} u(t)
  $$

produce:

$$
y(t) = (e^{-t} - e^{-2t}) u(t)
$$

---

#### Sustitución en la integral de convolución

$$
y(t) = \int_0^t e^{-2\tau} \cdot e^{-(t - \tau)} u(t - \tau) \, d\tau
$$

Sabemos que $ u(t - \tau) = 1 $ en el intervalo $ 0 \leq \tau \leq t $, entonces:

$$
y(t) = \int_0^t e^{-2\tau} \cdot e^{-(t - \tau)} \, d\tau = e^{-t} \int_0^t e^{-\tau} \, d\tau
$$

Calculamos la integral:

$$
\int_0^t e^{-\tau} \, d\tau = \left[ -e^{-\tau} \right]_0^t = -e^{-t} + 1
$$

Por lo tanto:

$$
y(t) = e^{-t} (1 - e^{-t}) = e^{-t} - e^{-2t}
$$

---

### ✅ Conclusión

Hemos comprobado manualmente que:

$$
x(t) * h(t) = (e^{-t} - e^{-2t}) u(t)
$$

lo cual coincide con la solución obtenida resolviendo la EDO.
